
<div style="text-align: center; line-height: 0; padding-top: 9px;">  
  <img src="/Volumes/cmidev/default/preventech/DBX_hack/compnovai_logo.PNG" alt="Image">  
</div> 

This notebook performs below tasks:
- Creation of self-managed vector search index on the embeddings table for faster retrival 
* Set up an endpoint for Vector Search.

* Store the embeddings and their metadata using the Vector Search.
- Zero-shot learning using an embeddings model and ranking of most similar claims to predict failure model bucket label for a test claim data
* Retrieve documents from the vector store using similarity search.
* In this notebook, we also attempted one-shot and few-shot learning of the embeddings model and prediction to compare accuracies among zero, one and few shot learning model approaches


<div style="text-align: center; line-height: 0; padding-top: 9px;">  
  <img src="/Volumes/cmidev/default/preventech/DBX_hack/CompNovai_step2.png" alt="Step2">  
</div> 


## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **14.3.x-cpu-ml-scala2.12 14.3.x-scala2.12**


**🚨 Important:** This demonstration relies on the resources established in the previous one. Please ensure you have completed the prior demonstration before starting this one.

In [0]:
%pip install -U --quiet mlflow==2.14.3 databricks-vectorsearch==0.40 transformers==4.43.3 "unstructured[pdf,docx]==0.14.10" langchain==0.2.11 langchain-community==0.2.10 pydantic==2.8.2 flashrank==0.2.8 pyspark==3.1.2 PyMuPDF accelerate
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.2.0 requires pydantic<2,>=1.8.1, but you have pydantic 2.8.2 which is incompatible.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%run ./_helper_functions

%md
## Create a "Self-Managed" Vector Search Index

Setting up a Databricks Vector Search index involves a few key steps. In this notebook, we created a vector search endpoint a vector search index** from a Delta table. 

In [0]:
# assign vs search endpoint by username
vs_endpoint_prefix = "vs_endpoint_ccims"
vs_endpoint_fallback = "vs_endpoint_ccims_fallback"
vs_endpoint_name = vs_endpoint_fallback #try this
print(f"Vector Endpoint name: {vs_endpoint_name}. In case of any issues, replace variable `vs_endpoint_name` with `vs_endpoint_fallback` in demos and labs.")

Vector Endpoint name: vs_endpoint_ccims_fallback. In case of any issues, replace variable `vs_endpoint_name` with `vs_endpoint_fallback` in demos and labs.


In [0]:
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c

vsc = VectorSearchClient(disable_notice=True)

# Define the endpoint name


In [0]:
if vs_endpoint_name not in [e["name"] for e in vsc.list_endpoints()["endpoints"]]:
    vsc.create_endpoint(name=vs_endpoint_name, endpoint_type="STANDARD")

In [0]:
vs_endpoint_name

'vs_endpoint_ccims_fallback'

# Check the status of the endpoint


In [0]:
wait_for_vs_endpoint_to_be_ready(vsc, vs_endpoint_name)
print(f"Endpoint named {vs_endpoint_name} is ready.")

Endpoint named vs_endpoint_ccims_fallback is ready.


### View the Endpoint

After the endpoint is created, you can view your endpoint on the [Vector Search Endpoints UI](#/setting/clusters/vector-search). Click on the endpoint name to see all indexes that are served by the endpoint.

# Connect Delta Table with Vector Search Endpoint

In [0]:
# the table we'd like to index

source_table_fullname = f"cmidev.cbu_ccims_poc.claim_text_embeddings"

# where we want to store our index
vs_index_fullname= f"cmidev.cbu_ccims_poc.claim_text_embeddings_index"
# create or sync the index
if not index_exists(vsc, vs_endpoint_name, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {vs_endpoint_name}...")
  vsc.create_delta_sync_index(
    endpoint_name=vs_endpoint_name,
    index_name=vs_index_fullname,
    source_table_name=source_table_fullname,
    pipeline_type="TRIGGERED", #Sync needs to be manually triggered
    primary_key="id",
    embedding_dimension=1024, #Match your model embedding size (gte)
    embedding_vector_column="embedding"
  )
else:
  # trigger a sync to update our vs content with the new data saved in the table
  vsc.get_index(vs_endpoint_name, vs_index_fullname).sync()

# let's wait for the index to be ready and all our embeddings to be created and indexed
wait_for_index_to_be_ready(vsc, vs_endpoint_name, vs_index_fullname)

In [0]:
#Test case 1 with known failure mode bucket
claim='"1608.0","27620","1693","687.0","X15 3 2017","ELTS","1033900"," MESSAGE:Service Support Center Cummins Inc, Receiving Location 35, 910 S Marr Road, Columbus, IN 47201 ****IMPORTANT**EPR**PART REQUEST - Review CUMMINS MATERIAL RETURN TICKET/s for specific part numbers to be returned **EPR Core Part***Goto QSOL Warranty Click on Request Core Return Process** ***Enhanced Parts Return Warning*** UPS: Visit https://row,ups,com/Default,aspx?Company=cumminsssc&LoginId=SSCship&Password=SSCpart1 to print your return label, FEDEX: 332426788 Important: Always print and include a copy of the parts return tag inside the box,FOLLOWING PARTS TO BE RETURNED:535130700 HOOK UP COMPUTER TO CHECKED CODES FINDING NONE ACTIVE STARTED TROUBLESHOOTING 687 AS IT HAD 7 COUNTS WITHIN THE LAST 25 HOURS , OPEN EDS AND FOLLOW PROMPTS 1, CHECKED FOR OTHER CODES FINDING NONE OF THE LISTED CODES PRESENT 2, REMOVED TURBO SPEED SENSOR AND INSPECTED FINDING NO DAMAGE 3, FOUND A GOOD 5V SIGNAL AT THE TURBO SPEED SENSOR HARNESS4, REMOVED INTAKE TEMP SENSOR FINDING NO DAMAGE BUT IT DID HAVE SOOT ACCUMULATION SO I CLEANED IT 5, REMOVED ECM CONNECTOR AND TURBO SPEED CONNECTOR AND CHECKED PINS TO THE ECM FINDING ALL WHERE AT 0,3OHMS IN SPEC 6, CHECKED GROUND TO ENGINE BLOCK AND FOUND IT WAS IN SPEC AT OL7, CHECKED PIN CONTINUITY WITH ALL OTHER PINS FINDING NO CONTINUITY WITH ANY OTHER PINS 8, CHECKED CALIBRATION AND FOUND THERE ARE MULTIPLE NEWER CALIBRATIONS AVAILABLE BUT NONE RELATED TO THESE CODES 9, EDS LEADS ME TO REPLACE THE TURBO SPEED SENSOR REPLACED SENSOR AND WENT FOR A ROAD TEST WITH NO CODES COMING BACK ALL GOOD AT THIS TIME EPR Part Number:535130700 Tracking Number:1z4r54w6880002738,CHECK ENGINE LIGHT YELLOWTurbocharger Speed SensorINDETERMINATE1084534 MONDAY 17JUN2019 11:27:31 AM"'

In [0]:
#Test case 2
claim = '{"OEM_CODE":1608,"DEALER":"99876","DISTR":"1692","FC":"2147.0,1897.0","ENGINE_NAME_DESC":"X15 3 2017","FAILCODE":"THAC","SHOPORDERNUM":"1473300","CLEANED_CORRECTION":" 6262020 6:23:34 AM 3025 customer called that cel is back on drove to cust locationBRENNER OIL CO 2147 3RD STNILES, MI 49120due to cel comming back on when truck put under load, hooked up computer and pulled codes again, active code again for vgt, fc 1897 this time, performed ts and found actuator failed, drained coolant and removed old actuator, insp turbo, good, installed and calabrated new one, refilled coolant, cleared all fault codes,performed regen, all temps and pressures look good, power washed engine road tested unit and no faults returned, road tested to confirm that cel didnt come on since this is the second trip out to customer for cel issue,CHECK ENGINE LIGHT YELLOWTURBOCHARGER ACTUATORFAULT LAMP23451DANKO MONDAY 29JUN2020 07:59:36 AM"}'

In [0]:
#Test case 3
#Failmode bucket: TSS - INTERMITTENT SIGNAL
claim_context='{"OEM_CODE":9095,"DEALER":"29597","DISTR":"3750","FC":"687.0","ENGINE_NAME_DESC":"X15 1 2017","FAILCODE":"ELTS","SHOPORDERNUM":"2979700","FAILURE_MODE_BUCKET":"TSS - INTERMITTENT SIGNAL","CLEANED_CORRECTION":" SRT190CQ overlap with 190CLCheck engine light on, Connected to Cummins and inspected codes, Fault code active for turbo speed sensor, Opened eds to troubleshoot, Inspected if any other fault code was triggering turbo speed code, Inspected sensor supply voltage, voltage was in spec, Inspect intake manifold pressure sensor for carbon build up, removed sensor to inspect, no build up found, Inspect for signal wire open circuit, Removed harness from ECM and ohmed signal to signal from sensor connector to ecm connector, That was less than 10 ohms, Inspected return for a open circuit, ohmed from sensor connector to ecm connector and that was also less than 10 ohms, Checked for turbo sensor shorted to ground, It was greater than 100k ohms, Inspected signal pin and ohmed out signal pin at ecm to inspect for a pin to pin short, All other pins on ecm connecter are not shorted to signal pin for turbo speed sensor connector, Checked ecm calibration history, Update is available but not relavent to active code, Advised replace turbo speed senor, Speed sensor ordered, Replaced speed sensor, EPR Part Number:535130700 Tracking Number:1Z155E540350198332,CHECK ENGINE LIGHT YELLOWTurbocharger Speed SensorINOPERATIVEMLEICHMAN FRIDAY 23FEB2018 09:08:13 AM"}'

In [0]:
#Test case 4
#ACT(E) LOOSE APS MAGNET
claim='{"OEM_CODE":2758,"DEALER":"19958","DISTR":"1693","FC":"1897.0","ENGINE_NAME_DESC":"B6.7 2017","FAILCODE":"THAC","SHOPORDERNUM":"5998900","FAILURE_MODE_BUCKET":"ACT(E) LOOSE APS MAGNET","CLEANED_CORRECTION":" drain and fill coolant replaced vgt actuator and calibrated, send on road test and run regen, EPR Part Number:549487800 Tracking Number:775883091309,CHECK ENGINE LIGHT YELLOWTurbocharger ActuatorDATA LINK19958BBAKER SUNDAY 04AUG2019 04:09:58 PM"}'

# Zero Shot Learning


In [0]:
import mlflow
from mlflow.deployments import get_deploy_client

def Predict_ZeroShotLearning(question):
    deploy_client = mlflow.deployments.get_deploy_client("databricks")  

    response = deploy_client.predict(endpoint="databricks-gte-large-en", inputs={"input": [question]})
    embeddings = [e["embedding"] for e in response.data]
   
    return embeddings

# One Shot Learning
In this modeling approach, we provided a prompt and context to the embeddings model.

In [0]:
import mlflow
from mlflow.deployments import get_deploy_client

def Predict_OneShotLearning(question):
    deploy_client = mlflow.deployments.get_deploy_client("databricks")
      
    prompt = "You are a diesel engine service assistant, who is expert in the turbochargers. You will be given a warranty claim text, along with other key details of the engine.  Here is an example of such claim:"+claim_context+" For this claim, the FAILURE MODE BUCKET is 'TSS - INTERMITTENT SIGNAL'.By reading those details, you need to predict failure mode bucket of the turbocharger."

    question_with_prompt = prompt+" " +question
    response = deploy_client.predict(endpoint="databricks-gte-large-en", inputs={"input": [question_with_prompt]})
    embeddings = [e["embedding"] for e in response.data]
    return embeddings

In [0]:
# few_shot_claim_context

In [0]:
claim_contexts= spark.sql("select DOCUMENT from cmidev.cbu_ccims_poc.ccims_claim_inspection_feature where FAILURE_MODE_BUCKET != 'Not Inspected' limit 200").toPandas()


In [0]:
display(claim_contexts)

DOCUMENT
"List( MESSAGE:Service Support Center Cummins Inc, Receiving Location 35, 910 S Marr Road, Columbus, IN 47201 ****IMPORTANT**EPR**PART REQUEST - Review CUMMINS MATERIAL RETURN TICKET/s for specific part numbers to be returned **EPR Core Part***Goto QSOL Warranty Click on Request Core Return Process** ***Enhanced Parts Return Warning*** UPS: Visit https://row,ups,com/Default,aspx?Company=cumminsssc&LoginId=SSCship&Password=SSCpart1 to print your return label, FEDEX: 332426788 Important: Always print and include a copy of the parts return tag inside the box,FOLLOWING PARTS TO BE RETURNED:535130700 HOOK UP COMPUTER TO CHECKED CODES FINDING NONE ACTIVE STARTED TROUBLESHOOTING 687 AS IT HAD 7 COUNTS WITHIN THE LAST 25 HOURS , OPEN EDS AND FOLLOW PROMPTS 1, CHECKED FOR OTHER CODES FINDING NONE OF THE LISTED CODES PRESENT 2, REMOVED TURBO SPEED SENSOR AND INSPECTED FINDING NO DAMAGE 3, FOUND A GOOD 5V SIGNAL AT THE TURBO SPEED SENSOR HARNESS4, REMOVED INTAKE TEMP SENSOR FINDING NO DAMAGE BUT IT DID HAVE SOOT ACCUMULATION SO I CLEANED IT 5, REMOVED ECM CONNECTOR AND TURBO SPEED CONNECTOR AND CHECKED PINS TO THE ECM FINDING ALL WHERE AT 0,3OHMS IN SPEC 6, CHECKED GROUND TO ENGINE BLOCK AND FOUND IT WAS IN SPEC AT OL7, CHECKED PIN CONTINUITY WITH ALL OTHER PINS FINDING NO CONTINUITY WITH ANY OTHER PINS 8, CHECKED CALIBRATION AND FOUND THERE ARE MULTIPLE NEWER CALIBRATIONS AVAILABLE BUT NONE RELATED TO THESE CODES 9, EDS LEADS ME TO REPLACE THE TURBO SPEED SENSOR REPLACED SENSOR AND WENT FOR A ROAD TEST WITH NO CODES COMING BACK ALL GOOD AT THIS TIME EPR Part Number:535130700 Tracking Number:1z4r54w6880002738,CHECK ENGINE LIGHT YELLOWTurbocharger Speed SensorINDETERMINATE1084534 MONDAY 17JUN2019 11:27:31 AM, 27620, 1693, X15 3 2017, ELTS, TSS - INTERMITTENT SIGNAL, 687.0, 1608.0, 1033900)"
"List( SRT190CQ overlap with 190CLCheck engine light on, Connected to Cummins and inspected codes, Fault code active for turbo speed sensor, Opened eds to troubleshoot, Inspected if any other fault code was triggering turbo speed code, Inspected sensor supply voltage, voltage was in spec, Inspect intake manifold pressure sensor for carbon build up, removed sensor to inspect, no build up found, Inspect for signal wire open circuit, Removed harness from ECM and ohmed signal to signal from sensor connector to ecm connector, That was less than 10 ohms, Inspected return for a open circuit, ohmed from sensor connector to ecm connector and that was also less than 10 ohms, Checked for turbo sensor shorted to ground, It was greater than 100k ohms, Inspected signal pin and ohmed out signal pin at ecm to inspect for a pin to pin short, All other pins on ecm connecter are not shorted to signal pin for turbo speed sensor connector, Checked ecm calibration history, Update is available but not relavent to active code, Advised replace turbo speed senor, Speed sensor ordered, Replaced speed sensor, EPR Part Number:535130700 Tracking Number:1Z155E540350198332,CHECK ENGINE LIGHT YELLOWTurbocharger Speed SensorINOPERATIVEMLEICHMAN FRIDAY 23FEB2018 09:08:13 AM, 29597, 3750, X15 1 2017, ELTS, TSS - INTERMITTENT SIGNAL, 687.0, 9095.0, 2979700)"
"List( drain and fill coolant replaced vgt actuator and calibrated, send on road test and run regen, EPR Part Number:549487800 Tracking Number:775883091309,CHECK ENGINE LIGHT YELLOWTurbocharger ActuatorDATA LINK19958BBAKER SUNDAY 04AUG2019 04:09:58 PM, 19958, 1693, B6.7 2017, THAC, ACT(E) LOOSE APS MAGNET, 1897.0, 2758.0, 5998900)"
"List( IMPORTANTEPRPART REQUEST Review CUMMINS MATERIAL RETURN TICKETs for specific part numbers to be returnedEPR Core ReturnGoto QSOL Warranty Click on Request Core Return Processcode 0687 followed EDS led to replacing turbo speed sensor, replaced cleared codes and all ok, EPR Part Number:535130700 Tracking Number:h9952818426,CHECK ENGINE LIGHT YELLOWTurbocharger Speed SensorINDETERMINATE1015600 SUNDAY 19AUG2018 09:30:17 AM, 16143, 1693, X15 3 2017, ELTS, TSS - CORROSION IN SENSOR BODY, 687.0, 14

# Generate context for few shot learning model by taking a few claim details

In [0]:

few_shot_claim_context=''
context_claims_list_temp = claim_contexts["DOCUMENT"].tolist()
context_claims_list = context_claims_list_temp[153:180]
# print("lenght:",len(context_claims_list))
claim_number =1
for doc in context_claims_list:
    # print(str(doc['DOCUMENT']).lower())
    few_shot_claim_context += "###  Claim Number: "+str(claim_number)+"### Claim Text: "+ str(doc).lower() +" ###     "
    claim_number+=1

failmodebuckets_list = spark.sql("select distinct(FAILURE_MODE_BUCKET) from cmidev.cbu_ccims_poc.ccims_claim_inspection_feature where FAILURE_MODE_BUCKET != 'Not Inspected'").toPandas()
failmodebuckets = failmodebuckets_list["FAILURE_MODE_BUCKET"].tolist()
total_failmodebuckets=''
for f in failmodebuckets:
    total_failmodebuckets += f.lower() +","


In [0]:
few_shot_claim_context

'###  Claim Number: 1### Claim Text: {\'oem_code\': 9147.0, \'dealer\': \'29476\', \'distr\': \'2991\', \'fc\': \'2387.0,1939.0\', \'engine_name_desc\': \'b6.7 2017\', \'failcode\': \'thac\', \'shopordernum\': \'7407000\', \'failure_mode_bucket\': \'turbine/nozzle fod\', \'cleaned_correction\': \' penalty pmu applied to the claim, srt1009b00 overlap with 1009g00perf diags fd code 2387, chkd, no associated faults, perf air handling perf tst, inspect turbo, fd turbo with fin contact, replaced turbo, while replacing turbo fd actuator arm binding  not swinging freely, replaced actuator, exhaust pipe clamp broke upon removal, replaced, reassembled all, cleared code  rd tst to verify repair, all ok, please note: this is a resubmitted claim, part was sent back previously tracking 1z1558090310128206, epr part number:560418800rx tracking number:276117915451,check engine light  yellowturbochargerinoperativeturbocharger actuatorinoperative21787cp thursday 28jul2022 01:50:43 pm ** high priority pa

In [0]:
# total_failmodebuckets

# Few Shot Learning
In this modeling approach, we provided an illustrative prompt to the model with an elaborated context compared to the one-shot learning model

In [0]:
#Few Shot Learning
import mlflow
from mlflow.deployments import get_deploy_client

def Predict_FewShotLearning(question):
    deploy_client = mlflow.deployments.get_deploy_client("databricks")
    prompt = "You are a diesel engine service assistant, who is an expert in the turbochargers. You will be given a warranty claim text, along with other key details of the engine and associated failure related to the turbocharger system, turbocharger assembly or turbo speed sensor.  Here are some examples of such claims, where failure mode bucket is followed by the keyword 'FAILURE_MODE_BUCKET:'. This failure mode bucket signifies real cause of the failure of turbocharger speed senor, or actuator or turbo assembly. The 'FAILCODE' field has values such as 'ELTS', which indicates Turbo Speed Senor related issues, 'THAC' indicates Turbocharger Actuator issues and 'THAS' indicates Turbocharger Assembly issues. The field called 'FC' stands for fault codes. These fault codes are associated with the FAILCODE field. 'distr' stands for distributor code, and cleaned_correction is a preprocessed and cleaned claim narrative text. Here are such examples of claim text:  "+ few_shot_claim_context+ "Standard FAILURE MODE BUCKET values are something like: " +total_failmodebuckets+" By reading those details, you need to predict failure mode bucket of the turbocharger." 
    question_with_prompt = prompt+" " +question
    response = deploy_client.predict(endpoint="databricks-gte-large-en", inputs={"input": [prompt,question]})
    embeddings = [e["embedding"] for e in response.data]

    return embeddings

In [0]:

from transformers import AutoModel, AutoTokenizer  
def LLMModel_FewShotLearning(question): 
    # bert_model_path = "/Volumes/cmidev/default/preventech/DBX_hack/google-bertbert-base-uncased"
    # model = AutoModel.from_pretrained(bert_model_path, local_files_only=True, revision="no_timm") 
    # tokenizer = AutoTokenizer.from_pretrained(model) 
    bert_model_path = "/Volumes/cmidev/default/preventech/DBX_hack/google-bertbert-base-uncased"
    from transformers import AutoTokenizer, AutoModelForCausalLM, Pipeline, AutoModelForSeq2SeqLM


    from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel,pipeline  

    tokenizer = AutoTokenizer.from_pretrained(bert_model_path, local_files_only=True, revision="no_timm")
    # model = AutoModel.from_pretrained(bert_model_path, local_files_only=True, revision="no_timm")
    model = AutoModelForQuestionAnswering.from_pretrained(bert_model_path, local_files_only=True, revision="no_timm")
    import torch  
  
    # Define the prompt and question  
    # prompt = "The movie was"  
    prompt = "You are a diesel engine service assistant, who is an expert in the turbochargers. You will be given a warranty claim text, along with other key details of the engine and associated failure related to the turbocharger system, turbocharger assembly or turbo speed sensor.  Here are some examples of such claims, where failure mode bucket is followed by the keyword 'FAILURE_MODE_BUCKET:'. This failure mode bucket signifies real cause of the failure of turbocharger speed senor, or actuator or turbo assembly. The 'FAILCODE' field has values such as 'ELTS', which indicates Turbo Speed Senor related issues, 'THAC' indicates Turbocharger Actuator issues and 'THAS' indicates Turbocharger Assembly issues. The field called 'FC' stands for fault codes. These fault codes are associated with the FAILCODE field. 'distr' stands for distributor code, and cleaned_correction is a preprocessed and cleaned claim narrative text. Here are such examples of claim text:  "+ few_shot_claim_context+ "Your answer should be from one of the allowed values for 'FAILURE MODE BUCKET' from the list: " +total_failmodebuckets+" By reading those details, you need to predict failure mode bucket of the turbocharger."
    # Tokenize the input using the BERT tokenizer  
    inputs = tokenizer(prompt, question, truncation=True, padding="max_length", max_length=512, return_tensors="pt")  
    # print(inputs)
    # Pass the input through the BERT model  
    outputs = model(**inputs)  
    print(outputs)
    # Use the pipeline method to perform question answering  
    qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)  
    result = qa_pipeline(question=question, context=prompt)  
    # print("result=======",result)
    # Extract the predicted label  
    predicted_label = result["answer"]  
  
    print("Predicted label:", predicted_label)  
    return predicted_label


# Get most similar documents using vector index on embeddings table


In [0]:
def GetSimilarDocuments(embeddings):
  # get similar 5 documents.
  results = vsc.get_index(vs_endpoint_name, vs_index_fullname).similarity_search(
    query_vector=embeddings[0],
    columns=["CLAIM_ID_SEQ", "DOCUMENT_CHUNK"],
    num_results=10)

  # format result to align with reranker lib format. 
  passages = []
  similar_claim_id=[]
  claim_proba=[]
  documents=[]
  # print("results:",results)
  for doc in results.get("result", {}).get("data_array", []):
      # print(doc)
      new_doc = {"CLAIM_ID_SEQ": doc[0], "text": doc[1], "probability": doc[2]}
      similar_claim_id.append(doc[0])
      claim_proba.append(doc[2])
      passages.append(new_doc)
      documents.append(doc[1].lower())

  # pprint(passages)
  # print("Similar claims are:",similar_claim_id)
  # print("Similar claims probabilty:",claim_proba)
  import pandas as pd
  df_sim_claims= pd.DataFrame()
  df_sim_claims['Similar_Claim_ID']=similar_claim_id
  df_sim_claims['Similar_Claim_Proba']=claim_proba
  df_sim_claims['Similar_Claim_Details']=documents
  df_sim_claims= df_sim_claims.sort_values(by='Similar_Claim_Proba', ascending=False)
  return df_sim_claims,documents,similar_claim_id
# display(df_sim_claims)

# Get label of most similar claims

In [0]:
def GetFailureModeBucket(similar_claim_id):
    # Assuming similar_claim_id is a list of IDs
    formatted_ids = ",".join([f"'{id}'" for id in similar_claim_id])
    query = f"""
    SELECT distinct(CLAIM_ID_SEQ), CLEANED_CORRECTION, FAILURE_MODE_BUCKET 
    FROM cmidev.cbu_ccims_poc.ccims_claim_inspection_feature 
    WHERE CLAIM_ID_SEQ IN ({formatted_ids})
    """
    claims_with_failmode = spark.sql(query)
    # display(claims_with_failmode)
    return claims_with_failmode#['FAILURE_MODE_BUCKET'].iloc[0]

# Re-ranking Search Results

For re-ranking the results, we used locally downloaded Bert Base Uncased Model of Google. By calling its Question Answering API, we ranked similar claims from most similar to least similar order.

In [0]:
bert_model_path = "/Volumes/cmidev/default/preventech/DBX_hack/google-bertbert-base-uncased"
from transformers import AutoTokenizer, AutoModelForCausalLM, Pipeline, AutoModelForSeq2SeqLM


from transformers import AutoTokenizer, AutoModelForQuestionAnswering
def GetSortedDocuments(question,documents):
    tokenizer = AutoTokenizer.from_pretrained(bert_model_path, local_files_only=True, revision="no_timm")
    model = AutoModelForQuestionAnswering.from_pretrained(bert_model_path, local_files_only=True, revision="no_timm")



    # Format the input for the tokenizer correctly
    # Combine question and each document into a single string
    inputs = tokenizer(
        [question + " " + doc for doc in documents],  # Combine question and document
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    import torch
    # Calculate the scores for each document:
    scores = torch.sum(start_scores * end_scores, dim=1)

    # Convert the scores to a 1D integer tensor:
    scores = scores.type(torch.IntTensor)

    # Sort the documents in descending order based on their scores:
    sorted_indices = torch.argsort(scores, descending=True)
    sorted_documents = [documents[i] for i in sorted_indices]

    # Print the sorted documents:
    # print(sorted_documents)
    return sorted_documents

2024-10-03 22:42:43.582955: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [0]:
def GetSimilarDocumentDetails(similar_claim_id,df_sim_claims,sorted_documents):
    get_index = []
    for document in sorted_documents:
        index = df_sim_claims[df_sim_claims['Similar_Claim_Details'] == document].index.tolist()
        # print(index)
        # for j in index:
        #     if j not in get_index:
        get_index.extend(index)
    claims=[]
    claims = ([similar_claim_id[i] for i in get_index])
    # print(claims)
    best_matching_doc_fail_mode = GetFailureModeBucket(claims)
    return best_matching_doc_fail_mode

# Prediction Label Extraction Approach 1 
Get most frequently predicted label in similar claims

In [0]:
#Approach with max count of fail modes

import numpy as np
from pyspark.sql.functions import col
def MostPopulatedFailModeBucket(best_matching_doc_fail_mode):
    fail_mode_counts = best_matching_doc_fail_mode.groupBy("FAILURE_MODE_BUCKET").count()
    max_count_fail_mode = fail_mode_counts.orderBy(col("count").desc()).first()["FAILURE_MODE_BUCKET"]
    # print(max_count_fail_mode)
    return max_count_fail_mode

# Prediction Label Extraction Approach 2 
Get label of top most matching claim

In [0]:
#Approach with top most matching document's fail mode
def TopMostMatchingFailModeBucket(best_matching_doc_fail_mode):

    fail_mode_counts = best_matching_doc_fail_mode.select("FAILURE_MODE_BUCKET").collect()[0]
    # print(fail_mode_counts)
    return fail_mode_counts

# Concatenate selected features

In [0]:
def ConcatenateFields(test_claims):
    from pyspark.sql.functions import concat_ws, struct
    
    df_final_updated = test_claims.withColumn("FC", concat_ws(",", "FC_1","FC_2","FC_3","FC_4","FC_5"))
    final_test_df = df_final_updated.withColumn("DOCUMENT", struct("OEM_CODE", "DEALER", "DISTR", "FC", "ENGINE_NAME_DESC", "FAILCODE", "SHOPORDERNUM",  "CLEANED_CORRECTION"))\
    .drop("FC",'FC 1', 'FC 2', 'FC 3', 'FC 4', 'FC 5')
    # display("-----",type(final_test_df))
    return final_test_df

# Remove unwanted labels

(This information was received from our Subject Matter Expert after we built this notebook, so it is not used in this notebook- For future use)

In [0]:
def RemoveUnwantedLabelExamples(df):
  unwanted_failmode_buckets = ['ACT(E) PENDING SUPPLIER ANALYSIS',
                               'TSS - PENDING SUPPLIER ANALYSIS',
                               'ACT(E) SUPPLIER ANALYSIS NOT CONDUCTED',
                               'UNKNOWN/INDETERMINATE',
                               'ACT(E) WRONG PART RETURNED',
                               'WRONG PART RETURNED',
                               'TSS - SUPPLIER ANALYSIS NOT CONDUCTED'
                               'TSS - WRONG PART RETURNED',
                               'ACT(E) UNKNOWN/INDETERMINATE',
                               'PENDING GROUP REVIEW',
                               'PENDING LAB RESPONSE',
                               'WRONG PART USED',
                               'ACT(E) WRONG PART USED',
                               'TSS - UNKNOWN/INDETERMINATE',
                               'TSS - WRONG PART USED',
                               'OUT OF WARRANTY']
  df_clean = df[~df['FAILURE_MODE_BUCKET'].isin(unwanted_failmode_buckets)]
  return df_clean

In [0]:
#Put everything together to call required functions on multiple claims
test_claims = spark.sql("select * from cmidev.cbu_ccims_poc.ccims_claim_inspection_feature where FAILURE_MODE_BUCKET != 'Not Inspected' limit 25")
final_test_df=ConcatenateFields(test_claims)
predicted_label_zh=[]
predicted_label_fh=[]
predicted_label_oh=[]

test_claims_list = final_test_df.select("DOCUMENT").collect()
# claim_id = test_claims.select("CLAIM_ID_SEQ").collect()
# actual_label = final_test_df.select("FAILURE_MODE_BUCKET").collect()
claim_id = [row.CLAIM_ID_SEQ for row in test_claims.select("CLAIM_ID_SEQ").collect()]
actual_label = [row.FAILURE_MODE_BUCKET for row in final_test_df.select("FAILURE_MODE_BUCKET").collect()]

print(test_claims_list)
for test_claim, claim_no, actual_l in zip(test_claims_list,claim_id,actual_label):
    # question = "What is predicted 'failure mode bucket' having below claim details: "+str(test_claim) +" Show probability of each failure mode bucket"
    # print(test_claim)
    # document_text = str(test_claim.DOCUMENT)
    # document_dict = test_claim.DOCUMENT.asDict()

    # Concatenate all values within the DOCUMENT struct into a single string
    # document_text = ", ".join([str(value) for value in document_dict.values()])

    # print(type(document_text))

    # question = "What is predicted 'failure mode bucket' having below claim details: " + test_claim + " Show probability of each failure mode bucket"
    document_text = str(test_claim.DOCUMENT)
    question = "What is predicted 'failure mode bucket' having below claim details: " + document_text + " Show probability of each failure mode bucket"
    print(document_text)

   
    embeddings_zeroshot = Predict_ZeroShotLearning(question)
    embeddings_fewshot= Predict_FewShotLearning(question)
    embeddings_oneshot= Predict_OneShotLearning(question)
    # embeddings = Predict_FewShotLearning(test_claim,question)
    # embeddings = Predict_ZeroShotLearning(test_claim,question)

    df_sim_claims_zh,documents_zh,similar_claim_id_zh = GetSimilarDocuments(embeddings_zeroshot)
    df_sim_claims_fh,documents_fh,similar_claim_id_fh = GetSimilarDocuments(embeddings_fewshot)
    df_sim_claims_oh,documents_oh,similar_claim_id_oh = GetSimilarDocuments(embeddings_oneshot)

    # Zero Shot Learning
    # claims_with_failmode =GetFailureModeBucket(similar_claim_id)
    sorted_documents_zh=GetSortedDocuments(question,documents_zh)
    best_matching_doc_fail_mode_zh=GetSimilarDocumentDetails(similar_claim_id_zh,df_sim_claims_zh,sorted_documents_zh)
    label_zh = (TopMostMatchingFailModeBucket(best_matching_doc_fail_mode_zh))
    
    # predicted_label.append(list(label[0].asDict().values())[0])
    predicted_label_zh.append(label_zh[0])
    print("Zero Shot Learning ### claim id:", claim_no, "Actual Label:",actual_l, "predicted Label:",label_zh )

    # Few Shot Learning
    # claims_with_failmode =GetFailureModeBucket(similar_claim_id)
    sorted_documents_fh=GetSortedDocuments(question,documents_fh)
    best_matching_doc_fail_mode_fh=GetSimilarDocumentDetails(similar_claim_id_fh,df_sim_claims_fh,sorted_documents_fh)
    label_fh = (TopMostMatchingFailModeBucket(best_matching_doc_fail_mode_fh))
    
    # predicted_label.append(list(label[0].asDict().values())[0])
    predicted_label_fh.append(label_fh[0])
    print("Few Shot Learning ### claim id:", claim_no, "Actual Label:",actual_l, "predicted Label:",label_fh )

    # One Shot Learning
    # claims_with_failmode =GetFailureModeBucket(similar_claim_id)
    sorted_documents_oh=GetSortedDocuments(question,documents_oh)
    best_matching_doc_fail_mode_oh=GetSimilarDocumentDetails(similar_claim_id_oh,df_sim_claims_oh,sorted_documents_oh)
    label_oh = (TopMostMatchingFailModeBucket(best_matching_doc_fail_mode_oh))
    
    # predicted_label.append(list(label[0].asDict().values())[0])
    predicted_label_oh.append(label_oh[0])
    print("One Shot Learning ### claim id:", claim_no, "Actual Label:",actual_l, "predicted Label:",label_oh )

    # test_claims = test_claims.withColumn("Prediction", (label))


[Row(DOCUMENT=Row(OEM_CODE=1608.0, DEALER='27620', DISTR='1693', FC='687.0', ENGINE_NAME_DESC='X15 3 2017', FAILCODE='ELTS', SHOPORDERNUM='1033900', CLEANED_CORRECTION=' MESSAGE:Service Support Center Cummins Inc, Receiving Location 35, 910 S Marr Road, Columbus, IN 47201 ****IMPORTANT**EPR**PART REQUEST - Review CUMMINS MATERIAL RETURN TICKET/s for specific part numbers to be returned         **EPR Core Part***Goto QSOL Warranty Click on Request Core Return Process** ***Enhanced Parts Return Warning***  UPS: Visit https://row,ups,com/Default,aspx?Company=cumminsssc&LoginId=SSCship&Password=SSCpart1 to print your return label,      FEDEX: 332426788   Important:  Always print and include a copy of the parts return tag inside the box,FOLLOWING PARTS TO BE RETURNED:535130700 HOOK UP COMPUTER TO CHECKED CODES FINDING NONE ACTIVE STARTED TROUBLESHOOTING 687 AS IT HAD 7 COUNTS WITHIN THE LAST 25 HOURS , OPEN EDS AND FOLLOW PROMPTS 1, CHECKED FOR OTHER CODES FINDING NONE OF THE LISTED CODES P

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Zero Shot Learning ### claim id: 17695581 Actual Label: TSS - INTERMITTENT SIGNAL predicted Label: Row(FAILURE_MODE_BUCKET='TSS - CORROSION IN SENSOR BODY')


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Few Shot Learning ### claim id: 17695581 Actual Label: TSS - INTERMITTENT SIGNAL predicted Label: Row(FAILURE_MODE_BUCKET='TROUBLE NOT FOUND')


...

Zero Shot Learning ### claim id: 17952675 Actual Label: ACT(E) LOOSE APS MAGNET predicted Label: Row(FAILURE_MODE_BUCKET='CONSEQUENTIAL DAMAGE')


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Few Shot Learning ### claim id: 17952675 Actual Label: ACT(E) LOOSE APS MAGNET predicted Label: Row(FAILURE_MODE_BUCKET='TROUBLE NOT FOUND')


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

One Shot Learning ### claim id: 17952675 Actual Label: ACT(E) LOOSE APS MAGNET predicted Label: Row(FAILURE_MODE_BUCKET='CONSEQUENTIAL DAMAGE')


In [0]:
# label_fh

In [0]:
# predicted_label_fh

In [0]:

import pandas as pd
result_df = pd.DataFrame()
result_df['claim_id'] = claim_id
result_df['actual_label'] = actual_label
result_df['predicted_label_zero'] = predicted_label_zh
result_df['predicted_label_one'] = predicted_label_oh
result_df['predicted_label_few'] = predicted_label_fh



display(result_df)

claim_id,actual_label,predicted_label_zero,predicted_label_one,predicted_label_few
17695581,TSS - INTERMITTENT SIGNAL,TSS - CORROSION IN SENSOR BODY,TSS - CORROSION IN SENSOR BODY,TROUBLE NOT FOUND
16295870,TSS - INTERMITTENT SIGNAL,TSS - BODY FAILURE,TSS - TROUBLE NOT FOUND,TROUBLE NOT FOUND
17848296,ACT(E) LOOSE APS MAGNET,ACT(E) CORROSION OF HOUSING AT CONNECTOR,TSS - TROUBLE NOT FOUND,TROUBLE NOT FOUND
16826341,TSS - CORROSION IN SENSOR BODY,TSS - INTERMITTENT SIGNAL,TSS - TROUBLE NOT FOUND,TROUBLE NOT FOUND
16997019,TSS - INTERMITTENT SIGNAL,TSS - INTERMITTENT SIGNAL,TSS - TROUBLE NOT FOUND,TROUBLE NOT FOUND
16225542,TSS - INTERMITTENT SIGNAL,TSS - INTERMITTENT SIGNAL,CONSEQUENTIAL DAMAGE,TROUBLE NOT FOUND
16225542,TSS - INTERMITTENT SIGNAL,WRONG PART RETURNED,TSS - TROUBLE NOT FOUND,TROUBLE NOT FOUND
18135554,CONSEQUENTIAL DAMAGE,CONSEQUENTIAL DAMAGE,TROUBLE NOT FOUND,TROUBLE NOT FOUND
17327126,ACT(E) CBA COMPONENT/PROCESS DEFECT,ACT(E) CBA COMPONENT/PROCESS DEFECT,TSS - INTERMITTENT SIGNAL,TROUBLE NOT FOUND
16776542,TSS - WELD PROCESS DEFECT,TSS - CORROSION IN SENSOR BODY,TSS - TROUBLE NOT FOUND,TROUBLE NOT FOUND


In [0]:
#Calculate accuracy zero shot
correct=0
for actual, predicted in zip(actual_label,predicted_label_zh):
    if actual==predicted:
        correct+=1
print("Accuracy of Zero Shot% =",correct*100/len(actual_label))

Accuracy of Zero Shot% = 56.0


In [0]:
#Calculate accuracy one shot
correct=0
for actual, predicted in zip(actual_label,predicted_label_oh):
    if actual==predicted:
        correct+=1
print("Accuracy of One Shot% =",correct*100/len(actual_label))

Accuracy of One Shot% = 4.0


In [0]:
#Calculate accuracy zero shot
correct=0
for actual, predicted in zip(actual_label,predicted_label_fh):
    if actual==predicted:
        correct+=1
print("Accuracy of Few Shot% =",correct*100/len(actual_label))

Accuracy of Few Shot% = 0.0


# Batch testing with Zero-Shot learning

In [0]:
#Put everything together to call required functions on multiple claims
test_claims = spark.sql("select * from cmidev.cbu_ccims_poc.ccims_claim_inspection_feature where FAILURE_MODE_BUCKET = 'Not Inspected' limit 25")
final_test_df=ConcatenateFields(test_claims)
predicted_label_zh=[]
predicted_label_fh=[]
predicted_label_oh=[]

test_claims_list = final_test_df.select("DOCUMENT").collect()
# claim_id = test_claims.select("CLAIM_ID_SEQ").collect()
# actual_label = final_test_df.select("FAILURE_MODE_BUCKET").collect()
claim_id = [row.CLAIM_ID_SEQ for row in test_claims.select("CLAIM_ID_SEQ").collect()]
actual_label = [row.FAILURE_MODE_BUCKET for row in final_test_df.select("FAILURE_MODE_BUCKET").collect()]

print(test_claims_list)
for test_claim, claim_no, actual_l in zip(test_claims_list,claim_id,actual_label):
    
    document_text = str(test_claim.DOCUMENT)
    question = "What is predicted 'failure mode bucket' having below claim details: " + document_text + " Show probability of each failure mode bucket"
    print(document_text)

   
    embeddings_zeroshot = Predict_ZeroShotLearning(question)
   
    df_sim_claims_zh,documents_zh,similar_claim_id_zh = GetSimilarDocuments(embeddings_zeroshot)
   
    # Zero Shot Learning
    # claims_with_failmode =GetFailureModeBucket(similar_claim_id)
    sorted_documents_zh=GetSortedDocuments(question,documents_zh)
    best_matching_doc_fail_mode_zh=GetSimilarDocumentDetails(similar_claim_id_zh,df_sim_claims_zh,sorted_documents_zh)
    label_zh = (TopMostMatchingFailModeBucket(best_matching_doc_fail_mode_zh))
    
    # predicted_label.append(list(label[0].asDict().values())[0])
    predicted_label_zh.append(label_zh[0])
    # print("Zero Shot Learning ### claim id:", claim_no, "Actual Label:",actual_l, "predicted Label:",label_zh )

    

[Row(DOCUMENT=Row(OEM_CODE=9095.0, DEALER='13618', DISTR='2853', FC='2387.0,4612.0', ENGINE_NAME_DESC='X15 3 2017', FAILCODE='THAS', SHOPORDERNUM='1189200', CLEANED_CORRECTION=' IMPORTANTEPRPART REQUEST  Review PRINT RETURN TAG for return instructions Go to QSOLWarrantyParts Return and refer to memoEDS DSBYV7445 and Insite led to diag of fault 2387, Could not perform regen analyzer test, DOC and DPF would not heat up, Fault 4612 became active, Replaced turbocharger, calibrated VGT, Steam cleaned engine, Completed DPF regen, Road test, Now have plenty of boost, EPR Part Number:545971100RX Tracking Number:774471444998,CHECK ENGINE LIGHT  YELLOWTURBOCHARGERINOPERATIVE13618KM FRIDAY 06AUG2021 01:40:12 PM **IMPORTANT**EPR**PART REQUEST - Review PRINT RETURN TAG for return instructions **Go to QSOL/Warranty/Parts Return and refer to memo**')), Row(DOCUMENT=Row(OEM_CODE=1608.0, DEALER='23038', DISTR='1693', FC='3361.0,3389.0,5177.0,5655.0,8156.0', ENGINE_NAME_DESC='X15 3 2017', FAILCODE='THAC

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Row(OEM_CODE=1608.0, DEALER='23038', DISTR='1693', FC='3361.0,3389.0,5177.0,5655.0,8156.0', ENGINE_NAME_DESC='X15 3 2017', FAILCODE='THAC', SHOPORDERNUM='1036300', CLEANED_CORRECTION=' SRT110CX00 OVERLAP WITH 110B100 SRT1009M00 OVERLAP WITH 110B100 SRT110CR00 OVERLAP WITH 110B100SRT110CX00 OVERLAP WITH 110B100SRT0807X00 OVERLAP WITH 110B100SRT110AU00 OVERLAP WITH 110B100SRT0808A00 OVERLAP WITH 110B100SRT1009M00 OVERLAP WITH 110B100SRT110CR00 OVERLAP WITH 110B100SRT190EW00 OVERLAP WITH 110B100SRT0808H00 OVERLAP WITH 110B100TS 5655 WITH EDS FOUND TURBO ACTUATOR FAILED WHILE REPLACING TURBO ACTUATOR HAD SIGNS OF COOLANT AT EGR COOLER OUTLET INSPECTED FOUND EGR COOLER LEAKING ,REMOVED TURBO STUDS BROKE OFF IN TURBO HAD TO DRILL OUT ,REMOVED AND REPLACED EGR COOLER AND TURBO ACTUATOR STARTED SCR TEST WATER PUMP STARTED TO LEAK FROM WEEP HOLE REMOVED ALTERNATOR AND AC COMPRESSOR RR WATER PUMP CORE ,CONTINUE TS SCR TEST FAILED INJSPECTED DOC CLEANED AND REINSTALLED OPEN CCR DATALOGGED REGEN A

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Row(OEM_CODE=76470.0, DEALER='10350', DISTR='1694', FC='5177.0', ENGINE_NAME_DESC='X15 3 2017', FAILCODE='THAC', SHOPORDERNUM='1052300', CLEANED_CORRECTION=' Removed 14-00T-00; not a condition to verify repair,  SRT1009M00 OVERLAP WITH 110AR00HOOK UP TO ECM, FOUND ACTIVE FAULT 5177 FOR VGT SOFTWARE, STARTED EDS, CHECKED FOR RELATED FAULTS, NONE ACTIVE, FAILED TURBO ACTUATOR, UNIT ALSO HAD EXHAUST LEAKS, PUT OUTRIGGERS OUT, LIFTED LADDER, AND LIFTED THE CAB, INSPECTED FOR EXHAUST LEAKS AND FOUND 6 LEAKING, DRAINED COOLANT, REMOVED THE EXHAUST MANIFOLD, MEASURED WARPAGE AND FOUND BOTH PIECES WARPED, INSTALLED NEW MANIFOLD WITH NEW GASKETS, REMOVED AND INSTALLED THE NEW TURBO ACTUATOR, CALIBRATED, REFILLED WITH COOLANT, CLEARED FAULTS, STARTED AND RAN UNIT, CHECKED FOR LEAKS, NONE FOUND, PUT UNIT INTO A REGEN, IT PASSED WITH NO ISSUES, STEAMED UNIT OFF, LOWERED CAB, LADDER, AND PUT IN THE OUTRIGGERS, REPAIRS COMPLETED, 99  LIFTLOWER CAB  LADDER ,5 EPR PART NUMBER:550118500RX TRACKING NUMB

...

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Row(OEM_CODE=1608.0, DEALER='23785', DISTR='1694', FC='999.0', ENGINE_NAME_DESC='X15 1 2017', FAILCODE='THAS', SHOPORDERNUM='3080700', CLEANED_CORRECTION=' SRT190CQ00 overlap with 190CL00CHECK ENGINE LAMP ON LOW POWER,CONN INSITE, LOGFILE, TS FC 687FOLLOW ALL STEPS 1 TO 8 RR NW TURBO SPEED SENSOR,C, CODE, ROAD TEST OK EPR Part Number:564324100 Tracking Number:1Z740Y379197316092,CHECK ENGINE LIGHT  YELLOWTURBOCHARGER SPEED SENSORAPPLICATION PROBLEM23785ROBIN FRIDAY 14JAN2022 07:45:46 AM **IMPORTANT**EPR**PART REQUEST - Review PRINT RETURN TAG for return instructions **Go to QSOL/Warranty/Parts Return and refer to memo**')


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Row(OEM_CODE=1442.0, DEALER='4409', DISTR='4409', FC='5177.0', ENGINE_NAME_DESC='X15 1 2017', FAILCODE='THAC', SHOPORDERNUM='2956800', CLEANED_CORRECTION=' unit loss power turbo speed sensor was not responding, code 5943 replaced turbo speed sensor, diag steps: INSITE - Use During Troubleshooting, Multimeter Usage (with Pin Check), Each ECM Connector - Measure (CMI), Engine - Diagnostic Road Test/Off-Highway Engine Verification,   Duplicate Admin SRT 00-901-00 on another claim from the same repair event  Duplicate Admin SRT 00-724-00 on another claim from the same repair event')


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [0]:
import pandas as pd
result_df = pd.DataFrame()
result_df['claim_id'] = claim_id
result_df['actual_label'] = actual_label
result_df['predicted_label_zero'] = predicted_label_zh
display(result_df)

claim_id,actual_label,predicted_label_zero
21831495,Not Inspected,TSS - INTERMITTENT SIGNAL
23402474,Not Inspected,TROUBLE NOT FOUND
24318846,Not Inspected,ACT(E) ELECTROLYTIC C26 FAILURE
20487208,Not Inspected,TSS - BODY CRACKED/BROKEN
24353339,Not Inspected,ACT(E) DAMAGE INDUCED (THERMAL OVERSTRESS)
21993046,Not Inspected,TSS - INTERMITTENT SIGNAL
22083484,Not Inspected,TSS - CORROSION IN SENSOR BODY
18645257,Not Inspected,ACT(E) SUPPLIER ANALYSIS NOT CONDUCTED
24334469,Not Inspected,TSS - INTERMITTENT SIGNAL
17665326,Not Inspected,UNKNOWN/INDETERMINATE


# Write prediction results into a delta table

In [0]:
result_df.write.mode("overwrite") \
  .saveAsTable("cmidev.cbu_ccims_poc.ccims_claim_inspection_mo_zeroshot_prediction")


## Conclusion

In this notebook,the initial step involved creating a Vector Search index, which required the establishment of a compute endpoint and the creation of an index that is synchronized with a source Delta table. Following this, we performed zero, one and few shot learning and ranking of results and extracted prediction using Approach 2: Get label of top most matching claim, ran this model on test claims and wrote the predictions into a delta table for consumption through UI.